In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
import gc

In [2]:
country_list =['NLD', 'FRA', 'ESP']

In [3]:
path_data_RCA = os.path.join('/workspace/workspace/ufo-prediction', 'demo', 'RCA_subset.csv')
rca_dt = pd.read_csv(path_data_RCA, encoding='latin1')

In [4]:
def create_geodataframe(df, lat_col, lon_col):
    df['geometry'] = df.apply(lambda row: Point(row[lon_col], row[lat_col]), axis=1)
    return gpd.GeoDataFrame(df, geometry='geometry')

# Identifying match type
def match_type(row):
    if pd.isna(row['index_right0']):
        return 'no match'
    elif round(row['Lat_nb'], 5) == round(row['lat'], 5) and round(row['Lon_nb'], 5) == round(row['lon'], 5):
        return 'exact match'
    elif round(row['Lat_nb'], 4) == round(row['lat'], 4) and round(row['Lon_nb'], 4) == round(row['lon'], 4):
        return 'match on 10m accuracy'
    else:
        return 'match on 50m accuracy'

In [5]:
gdf_rca = create_geodataframe(rca_dt, 'Lat_nb', 'Lon_nb')
# Setting coordinate system (assuming WGS84)
gdf_rca.set_crs(epsg=4326, inplace=True)
# Transform the CRS to UTM zone 32N (EPSG:32632) so that distances are in meters
gdf_rca_n = gdf_rca.to_crs(epsg=32632)

In [6]:
for ctr in country_list:
    print(ctr)
    # read in the data
    df = pd.read_pickle('/workspace/workspace/ufo-prediction/demo/df-'+ctr+'.pkl')
    # drop the geometry column
    df = df.drop(columns=['geometry'])
    # convert to geodataframe
    gdf_ufo = create_geodataframe(df, 'lat', 'lon')
    # Setting coordinate system (assuming WGS84)
    gdf_ufo.set_crs(epsg=4326, inplace=True)
    # Transform the CRS to UTM zone 32N (EPSG:32632) so that distances are in meters
    gdf_ufo_n = gdf_ufo.to_crs(epsg=32632)
    # Spatial join - finds matches within 110 meters
    matches = gpd.sjoin_nearest(gdf_rca_n, gdf_ufo_n, distance_col='distances', how='left', max_distance=110)
    # add match type
    matches['match_type'] = matches.apply(match_type, axis=1)
    # Add a new column 'BldLenProxy' as the square root of 'FootprintArea'
    matches['BldLenProxy'] = np.sqrt(matches['FootprintArea'])
    # Set 'match_type' to 'no match' where 'BldLenProxy' is less than or equal to 'distances'
    matches.loc[matches['BldLenProxy'] <= matches['distances'], 'match_type'] = 'no match'
    # Filter out the rows where 'match_type' is 'no match'
    matches = matches[matches['match_type'] != 'no match']
    # rename the country column
    matches.rename(columns={'country_left': 'country'}, inplace=True)
    # save the data
    matches.to_csv('/workspace/workspace/ufo-prediction/demo/rca-ufo-merge'+ctr+'.csv', index=False)
    # free up memory
    del df, gdf_ufo, gdf_ufo_n, matches
    # collect garbage
    gc.collect()

NLD
FRA
ESP


In [7]:
combined_df = pd.DataFrame()
for ctr in country_list:
    print(ctr)
    # read in the data
    df = pd.read_csv('/workspace/workspace/ufo-prediction/demo/rca-ufo-merge'+ctr+'.csv', encoding='latin1')
    print(f"Number of rows before appending: {df.shape[0]}")
    # append country dataset to combined_df
    combined_df = pd.concat([combined_df, df], ignore_index=True)
    # Record the number of rows after appending and before removing duplicates
    rows_after_append = combined_df.shape[0]
    # Remove duplicates
    combined_df.drop_duplicates(inplace=True)
    # Record the number of rows after removing duplicates
    rows_after_duplicates_removed = combined_df.shape[0]
    # Calculate and print the number of duplicates removed
    duplicates_removed = rows_after_append - rows_after_duplicates_removed
    if duplicates_removed > 0:
        print(f"Removed {duplicates_removed} duplicate rows.")
    # Identify columns with all NA values
    all_na_columns = combined_df.columns[combined_df.isna().all()]
    if not all_na_columns.empty:
        print("Columns with all NA values:", all_na_columns.tolist())
    # free up memory
    del df
    # collect garbage
    gc.collect()

NLD
Number of rows before appending: 4560
Columns with all NA values: ['Unnamed: 0.1', 'type_source', 'floors', 'type']
FRA
Number of rows before appending: 1100
Columns with all NA values: ['Unnamed: 0.1']
ESP
Number of rows before appending: 1747


In [8]:
if 'residential_type' in combined_df.columns:
    na_share = combined_df['residential_type'].isna().mean()
    print(f"Share of NA values in 'residential_type': {na_share:.2f}")
else:
    print("Column 'residential_type' does not exist.")

Share of NA values in 'residential_type': 0.00


In [9]:
combined_df.rename(columns={'country_left': 'country'}, inplace=True)
if 'country' in combined_df.columns:
    na_share = combined_df['country'].isna().mean()
    print(f"Share of NA values in 'country': {na_share:.2f}")
else:
    print("Column 'country' does not exist.")

Share of NA values in 'country': 0.00


In [ ]:
combined_df.rename(columns={'country_left': 'country'}, inplace=True)
if 'country' in combined_df.columns:
    na_share = combined_df['country'].isna().mean()
    print(f"Share of NA values in 'country': {na_share:.2f}")
else:
    print("Column 'country' does not exist.")

In [10]:
combined_df.to_csv('/workspace/workspace/ufo-prediction/demo/rca-ufo-merge_ALL.csv', index=False)

In [ ]:
del combined_df, gdf_rca, gdf_rca_n, rca_dt
gc.collect()